# GP datasets with zero noise_var + NNs


In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/0305_gp_nn_noiseless/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

### Measures of shallow relu neural networks

In [ ]:
ms_nn = pd.read_feather("measures.feather")
ms_nn = smooth.analysis.expand_dataset_columns(ms_nn)
smooth.analysis.remove_constant_columns(ms_nn, verbose=True)
print("Before removal:", len(ms_nn))
ms_nn = ms_nn.loc[np.isfinite(ms_nn["path_length_f_test"])]
print("After removal:", len(ms_nn))
ms_nn.head()

In [ ]:
measure_cols = ["train_loss", "test_loss", "path_length_f", "path_length_d", "weights_rms"]

for measure in measure_cols:
    IPython.display.display(IPython.display.Markdown("### {}".format(measure)))
    if True:
        ms1 = ms_nn[(ms_nn["hidden_size"] == 64)]
        grid = sns.relplot(
            data=ms1,
            x="samples_train",
            y=measure,
            hue="init_scale",
            style="learning_rate",
            col="dim",
            col_wrap=3,
            kind="line",
            palette=smooth.analysis.make_palette(ms1["init_scale"].unique()),
        )
        ax = grid.axes[0] #[0]

        ax.set_xscale("log")
        if "loss" in measure or True:
            ax.set_yscale("log")
        plt.show()

In [ ]:
ms_gp = smooth.analysis.compute_or_load_df(
    lambda: smooth.analysis.get_gp_measures(ms_nn["dataset"].unique()),
    "measures_gp.feather",
    always_compute=False,
)

In [ ]:
def add_normalized_cols(ms):
    ms["plc"] = ms["path_length_f_test"] / ms["path_length_f_test_bound"]
    ms["plct"] = ms["path_length_f_train"] / ms["path_length_f_train_bound"]
    return ms

add_normalized_cols(ms_gp)

ms_bound = (ms_gp
    .drop(columns=["path_length_f_test", "path_length_f_train", "loss_train", "loss_test"])
    .rename(columns={
        "path_length_f_test_bound": "path_length_f_test",
        "path_length_f_train_bound": "path_length_f_train",
    })
)

In [ ]:
ms_gp

In [ ]:
def plot_compare(groups, filter_f: None):
    filter_f = filter_f or (lambda df: df)
    l = []
    for group_name, group in groups:
        for name, ms_cur in group:
            ms_cur = ms_cur.copy()
            ms_cur.loc[:, "source"] = name
            ms_cur.loc[:, "group"] = group_name
            l.append(ms_cur)

    ms_all = pd.concat(l, sort=False)
    ms_all = filter_f(ms_all)
    
    for measure in ["loss_train", "loss_test", "path_length_f_test", "path_length_f_train"]:
        grid = sns.relplot(
            data=ms_all,
            x="samples_train",
            y=measure,
            hue="source",
#             style="init_scale",
            col="dim",
            col_wrap=3,
            kind="line",
        )
        ax = grid.axes[0]#[0]
        ax.set_xscale("log")
        if measure in ["loss_train", "loss_test",
                      "path_length_f_train", "path_length_f_test",
                      ]:
            ax.set_yscale("log")
#         if measure in ["path_length_f"]:
#             ax.set_ylim(0.03, 30)
        plt.show()

nn_group = []
for init in sorted(ms_nn["init_scale"].unique()):
#     for lr in sorted(ms_nn["learning_rate"].unique()):
    lr = (0.01 / init).round(5)
    nn_group.append((
        "nn, lr={:.3f}, is={:.3f}".format(lr, init),
        ms_nn.loc[
            (ms_nn["hidden_size"] == 64) &
            (ms_nn["init_scale"] == init) &
            (ms_nn["learning_rate"] == lr)
        ],
    ))
nn_group.append(("gp", ms_gp))
# nn_group.append(("gp_noiseless", ms_gp2))
nn_group.append(("bound", ms_bound))

def filter_f(ms):
    return ms.loc[
#         (ms["dim"] == dim)
#         & (ms_all["seed"] == 1)
        (ms["lengthscale"] == ms["dim"])
        & (ms["disjoint"] == 1)
    ]

for dim in sorted(ms_nn["dim"].unique())[:1]:
    display(IPython.display.Markdown("### dim = {}".format(dim)))
    plot_compare([("nn", nn_group)], filter_f)

In [ ]:
def f(**kwargs):
    kwargs["a"] = 3

d = dict(a=1)
print(d)
f(**d)
print(d)